## PROJET GLO-4027
Prédiction éléction

## Lecture des données

In [5]:
import numpy as np
import pandas as pd

In [6]:
df=pd.read_csv("./../CES19.csv",parse_dates=['cps19_StartDate'])
dfTest=df[:2000]

In [3]:
#Pour tester les algo avec moins de données
df=dfTest
df.isnull().sum()

Unnamed: 0                            0
cps19_StartDate                       0
cps19_EndDate                         0
cps19_ResponseId                      0
cps19_consent                         0
                                   ... 
pes19_inattentive                  1156
cps19_weight_general_all              4
cps19_weight_general_restricted     314
pes19_weight_general_all           1157
pes19_weight_general_restricted    1220
Length: 621, dtype: int64

In [7]:
attributs=np.array(df.columns)

In [8]:
dfTrain=df[ (pd.isna(df['cps19_votechoice'])==False) | (pd.isna(df['cps19_votechoice_pr'])==False)
           |(pd.isna(df['cps19_vote_unlikely'])==False) | (pd.isna(df['cps19_vote_unlike_pr'])==False)
           |(pd.isna(df['cps19_v_advance'])==False)]
dfPredire=df[ (pd.isna(df['cps19_votechoice'])==True) & (pd.isna(df['cps19_votechoice_pr'])==True)
           &(pd.isna(df['cps19_vote_unlikely'])==True) & (pd.isna(df['cps19_vote_unlike_pr'])==True)
           &(pd.isna(df['cps19_v_advance'])==True)]
len(dfTrain)/len(df)
len(dfTrain)

32937

## ANALYSE statistique des données

% de data par attributs

In [9]:
def pourcentage_par_attributs(dataframe,attribut):
    maskNaN=np.array(dataframe[attribut].isna())
    compteur = np.count_nonzero(maskNaN == False)
    return compteur/len(maskNaN)

In [10]:
pourcentage_response=[]
for attribut in attributs :
    pourcentage_response.append([attribut,pourcentage_par_attributs(dfTrain,attribut)])
pourcentage_response

[['Unnamed: 0', 1.0],
 ['cps19_StartDate', 1.0],
 ['cps19_EndDate', 1.0],
 ['cps19_ResponseId', 1.0],
 ['cps19_consent', 1.0],
 ['cps19_citizenship', 1.0],
 ['cps19_yob', 1.0],
 ['cps19_yob_2001_age', 0.007468804080517351],
 ['cps19_gender', 1.0],
 ['cps19_province', 1.0],
 ['cps19_education', 1.0],
 ['cps19_demsat', 1.0],
 ['cps19_imp_iss', 0.8107599356346965],
 ['cps19_imp_iss_party', 0.8111546285332605],
 ['cps19_imp_iss_party_7_TEXT', 0.014785803200048577],
 ['cps19_imp_loc_iss', 0.35920089868536903],
 ['cps19_imp_loc_iss_p', 0.3593830646385524],
 ['cps19_imp_loc_iss_p_7_TEXT', 0.006649057291192276],
 ['cps19_interest_gen_1', 0.9372438291283359],
 ['cps19_interest_elxn_1', 0.9409478701763974],
 ['cps19_v_likely', 0.9646294440902329],
 ['cps19_v_likely_pr', 0.035370555909767135],
 ['cps19_votechoice', 0.8628897592373319],
 ['cps19_votechoice_7_TEXT', 0.005495339587697726],
 ['cps19_votechoice_pr', 0.03518838995658378],
 ['cps19_votechoice_pr_7_TEXT', 9.108297659167502e-05],
 ['cps19

In [11]:
#Moyenne du pourcentage de response par attribut
mean=0
for attr in pourcentage_response :
    mean+=attr[1]
mean/len(pourcentage_response)

0.31378339666362365

Fusionner attribut :

In [12]:
def fusion_attribut_vote(attributs_fusion,attribut_name):
    #priorité au premiers attributs de attributs_fusion
    erase=0
    tab=np.array(dfTrain[attributs_fusion])
    maskNaN=np.array(dfTrain[attributs_fusion].isna())
    new_tab=[]
    for i in range(len(tab)):
        new_tab.append(np.nan)
        if np.count_nonzero(maskNaN[i]==False)>=2 :
            if tab[i][3]!=tab[i][6] :
                if tab[i][3]=="Don't know/ Prefer not to answer":
                    maskNaN[i][3]=True
                else :
                    erase+=np.count_nonzero(maskNaN[i]==False)-1
                    print(tab[i])
        for j in range(len(tab[i])):
            if not maskNaN[i][j] :
                new_tab[-1]=tab[i][j]
                continue
    dfNan = dfTrain.assign(vote_choice = new_tab)
    print(erase)
    return dfNan

In [13]:
dfTrain=fusion_attribut_vote(['cps19_votechoice','cps19_votechoice_pr','cps19_vote_unlikely','cps19_vote_unlike_pr','cps19_v_advance'],'vote_choice')
dfTrain['vote_choice']

0


0                             Green Party
1        Don't know/ Prefer not to answer
2                           Liberal Party
3                      Conservative Party
4        Don't know/ Prefer not to answer
                       ...               
37816                                 ndp
37817                       Liberal Party
37818                         Green Party
37820                                 ndp
37821    Don't know/ Prefer not to answer
Name: vote_choice, Length: 32937, dtype: object

In [14]:
pourcentage_par_attributs(dfTrain,'vote_choice')

1.0

Séparation par parti

In [15]:
dfAnother=dfTrain[dfTrain['vote_choice']=='Another party (please specify)']
dfBlocQuebecois=dfTrain[dfTrain['vote_choice']=='Bloc Qu<e9>b<e9>cois']# Bloc Québécois
dfConservative=dfTrain[dfTrain['vote_choice']=='Conservative Party']# • Conservative Party
dfNo=dfTrain[dfTrain['vote_choice']=="Don't know/ Prefer not to answer"]# • Don't know/ Prefer not to answer
dfGreen=dfTrain[dfTrain['vote_choice']=='Green Party']# • Green Party
dfLiberal=dfTrain[dfTrain['vote_choice']=='Liberal Party']# • Liberal Party
dfNdp=dfTrain[dfTrain['vote_choice']=='ndp']# • NDP
dfPeople=dfTrain[dfTrain['vote_choice']=="People's Party"]# • People's Party

(len(dfAnother)+len(dfBlocQuebecois)+len(dfConservative)+len(dfNo)+len(dfGreen)+len(dfLiberal)+len(dfNdp)+len(dfPeople))/len(dfTrain)

1.0

In [16]:
def coherence_attr(dfVote,attribut,confiance):
    val_attr={"nan":0}
    val_rep={}
    
    for elmt in dfVote[attribut]:
        if elmt in val_attr :
            val_attr[elmt]+=1
        if pd.isna(elmt):
            val_attr["nan"]+=1
        else :
            val_attr[elmt]=1
    for val in val_attr :
        size=len(dfTrain[dfTrain[attribut]==val])
        if size > 5 :
            if val_attr[val]/size >= confiance :
                val_rep[val]= val_attr[val]/size
    return val_rep

In [17]:
def coherence_parti(dfVote,confiance):
    confiance_parti={}
    for attr in dfVote.columns :
        if attr not in ['cps19_votechoice','cps19_votechoice_pr','cps19_vote_unlikely','cps19_vote_unlike_pr','cps19_v_advance']:
            dico=coherence_attr(dfVote,attr,confiance)
            if dico!={} :
                confiance_parti[attr]=dico
        
    return confiance_parti

In [18]:
coherence_parti(dfNdp,0.15)

KeyboardInterrupt: 

Age

In [19]:
compte = [0,0,0,0]
for elmt in dfTrain['cps19_yob']:
    if elmt <= 1959 :
        compte[3]+=1
    elif 1959 < elmt <= 1979 :
        compte[2]+=1
    elif 1979 < elmt <= 1994 :
        compte[1]+=1
    elif 1994 < elmt :
        compte[0]+=1

compte

[2456, 8733, 11495, 10253]

In [20]:
# cps19_yob_norm{ np.nan : 0 ; 1994< : 0.25 ; 1979<.<=1994 : 0.5 ; 1959<.<=1979 : 0.75 ; .<=1959:1 ;
norm_val_yob=[]
for elmt in dfTrain['cps19_yob']:
    if elmt <= 1959 :
        norm_val_yob.append(1)
    elif 1959 < elmt <= 1979 :
        norm_val_yob.append(0.75)
    elif 1979 < elmt <= 1994 :
        norm_val_yob.append(0.5)
    elif 1994 < elmt :
        norm_val_yob.append(0.25)
    else :
        norm_val_yob.append(0)
len(norm_val_yob)

32937

In [21]:
def analyse_attr(df,attr):
    valeurs={}
    for elmt in df[attr]:
        if elmt not in valeurs :
            valeurs[elmt]=0
        else :
            valeurs[elmt]+=1
    return valeurs

In [22]:
rep={}
for i in range(11,20):
    rep[attributs[i]]=analyse_attr(dfTrain,attributs[i])
rep

{'cps19_demsat': {'Fairly satisfied': 18599,
  "Don't know/ Prefer not to answer": 698,
  'Very satisfied': 5085,
  'Not very satisfied': 6948,
  'Not at all satisfied': 1602},
 'cps19_imp_iss': {'environnement': 208,
  nan: 6232,
  'strengthening foreign relations': 0,
  'Taxation and Government Spending': 0,
  'Immigration Issues': 0,
  'Climate Change': 244,
  'Minimum Wage': 0,
  'Climate change/environmental issues': 1,
  'climate related issues': 0,
  'economic opportunities to young people, specifically reducing barriers to owning property.': 0,
  'Making sure youth voices are heard and our problems are assessed': 0,
  'Carbon Tax': 10,
  'ABORTION RIGHTS': 0,
  'immigration': 138,
  'Education Funding and Carbon tax': 0,
  'cost of living': 101,
  'climate change': 327,
  'Issues on  the environment': 0,
  'Education; Student Funding': 0,
  'Economy': 843,
  'Environmental issues/climate change': 2,
  'education and environment': 1,
  'Climate change': 816,
  'Osap funding': 0,

In [23]:
dfGlobal=[dfAnother,dfBlocQuebecois,dfConservative,dfNo,dfGreen,dfLiberal,dfNdp,dfPeople]
partyName=["Another","BlocQuebecois","Conservative","No","Green","Liberal","Ndp","People"]

In [24]:
def importance_valeur(attr):
    pourcentage={"Another":0 ,
                 "BlocQuebecois":0,
                 "Conservative":0,
                 "No":0,
                 "Green":0,
                 "Liberal":0,
                 "Ndp":0,
                 "People":0}
    valeurs=[]
    for i in range(len(dfGlobal)) :
        pourcentage[partyName[i]]=analyse_attr(dfGlobal[i],attr)
        for val in pourcentage[partyName[i]].keys() :
            if val not in valeurs :
                valeurs.append(val)
    return pourcentage,valeurs

In [25]:
attribut='cps19_demsat'
pourcentage,valeurs=importance_valeur(attribut)
rep=[["Valeur"]]
for party in partyName :
    rep[0].append(party)
rep[0].append("Total")

for val in valeurs :
    decompte=[val+" :"]
    lTotal=0
    for party in partyName :
        try :
            decompte.append(pourcentage[party][val])
            lTotal+=pourcentage[party][val]
        except :
            decompte.append(decompte.append(0))
    decompte.append(lTotal)
    rep.append(decompte)
rep

[['Valeur',
  'Another',
  'BlocQuebecois',
  'Conservative',
  'No',
  'Green',
  'Liberal',
  'Ndp',
  'People',
  'Total'],
 ['Fairly satisfied :', 98, 838, 4759, 2850, 1311, 5868, 2622, 246, 18592],
 ['Not very satisfied :', 71, 363, 2237, 1313, 729, 840, 1161, 227, 6941],
 ['Very satisfied :', 11, 195, 1072, 497, 304, 2480, 437, 82, 5078],
 ["Don't know/ Prefer not to answer :", 4, 21, 163, 219, 84, 95, 92, 13, 691],
 ['Not at all satisfied :', 28, 75, 639, 257, 178, 86, 242, 90, 1595]]

cps19_imp_iss 18589/32937  ['cps19_imp_iss_party', 0.8111546285332605]

In [26]:
dfTrain[dfTrain['cps19_imp_iss_party']==dfTrain['vote_choice']][['cps19_imp_iss_party','vote_choice']]

,cps19_imp_iss_party,vote_choice
0,Green Party,Green Party
3,Conservative Party,Conservative Party
4,Don't know/ Prefer not to answer,Don't know/ Prefer not to answer
9,Liberal Party,Liberal Party
14,Liberal Party,Liberal Party
...,...,...
37811,Bloc Qu<e9>b<e9>cois,Bloc Qu<e9>b<e9>cois
37815,Bloc Qu<e9>b<e9>cois,Bloc Qu<e9>b<e9>cois
37817,Liberal Party,Liberal Party
37818,Green Party,Green Party


cps19_imp_loc_iss_p 7752 ['cps19_imp_loc_iss_p', 0.3593830646385524],

In [27]:
dfTrain[dfTrain['cps19_imp_loc_iss_p']==dfTrain['vote_choice']][['cps19_imp_loc_iss_p','vote_choice']]

,cps19_imp_loc_iss_p,vote_choice
10,ndp,ndp
19,ndp,ndp
27,Liberal Party,Liberal Party
32,Conservative Party,Conservative Party
42,Liberal Party,Liberal Party
...,...,...
37777,ndp,ndp
37805,Bloc Qu<e9>b<e9>cois,Bloc Qu<e9>b<e9>cois
37806,Liberal Party,Liberal Party
37811,Bloc Qu<e9>b<e9>cois,Bloc Qu<e9>b<e9>cois


 ['cps19_interest_gen_1', 0.9372438291283359],
 ['cps19_interest_elxn_1', 0.9409478701763974],

In [28]:
dfTrain['cps19_fed_gov_sat']

0        Not very satisfied
1        Not very satisfied
2          Fairly satisfied
3        Not very satisfied
4        Not very satisfied
                ...        
37816    Not very satisfied
37817      Fairly satisfied
37818    Not very satisfied
37820      Fairly satisfied
37821      Fairly satisfied
Name: cps19_fed_gov_sat, Length: 32937, dtype: object

In [70]:
def gatherPartyAttribute(attribut,attributName,newAttrName):
    newAttrTab=[]
    for index, row in dfTrain[attr].iterrows() :
        newAttrTab.append(attributName[np.argmax(row)])
    
    dfTrain[newAttrName]=newAttrTab
    
    newAttrTab=[]
    for index, row in dfPredire[attr].iterrows() :
        newAttrTab.append(attributName[np.argmax(row)])
    
    dfPredire[newAttrName]=newAttrTab
    return dfTrain[dfTrain[newAttrName]==dfTrain['vote_choice']][[newAttrName,'vote_choice']].shape

def getMinPartyAttribute(attribut,attributName,newAttrName):
    newAttrTab=[]
    for index, row in dfTrain[attr].iterrows() :
        newAttrTab.append(attributName[np.argmin(row)])
    
    dfTrain[newAttrName]=newAttrTab
    
    newAttrTab=[]
    for index, row in dfPredire[attr].iterrows() :
        newAttrTab.append(attributName[np.argmin(row)])
    
    dfPredire[newAttrName]=newAttrTab

In [72]:
#'cps19_party_rating'
attr=['cps19_party_rating_23', 'cps19_party_rating_24', 'cps19_party_rating_25', 'cps19_party_rating_26', 'cps19_party_rating_27', 'cps19_party_rating_28']
attrNameParty= ['Liberal Party','Conservative Party','ndp','Bloc Québécois','Green Party',"People's Party"]

gatherPartyAttribute(attr,attrNameParty,'cps19_party_rating_max')

C:\Users\Jean\AppData\Local\Temp/ipykernel_18444/1029237248.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfPredire[newAttrName]=newAttrTab


(21730, 2)

In [73]:
#'cps19_party_rating_min'
attr=['cps19_party_rating_23', 'cps19_party_rating_24', 'cps19_party_rating_25', 'cps19_party_rating_26', 'cps19_party_rating_27', 'cps19_party_rating_28']
attrNameParty= ['Liberal Party','Conservative Party','ndp','Bloc Québécois','Green Party',"People's Party"]

getMinPartyAttribute(attr,attrNameParty,'cps19_party_rating_min')

C:\Users\Jean\AppData\Local\Temp/ipykernel_18444/1029237248.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfPredire[newAttrName]=newAttrTab


In [31]:
#'cps19_lead_rating'
attr=['cps19_lead_rating_23', 'cps19_lead_rating_24', 'cps19_lead_rating_25', 'cps19_lead_rating_26', 'cps19_lead_rating_27', 'cps19_lead_rating_28']
attrNamePeople=['Justin Trudeau','Andrew Scheer','Jagmeet Singh','Yves-François Blanchet','Elizabeth May','Maxime Bernier']

gatherPartyAttribute(attr,attrNamePeople,'cps19_lead_rating_max')

C:\Users\Jean\AppData\Local\Temp/ipykernel_18444/2886697018.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfPredire[newAttrName]=newAttrTab


(0, 2)

In [74]:
#'cps19_lead_rating_min'
attr=['cps19_lead_rating_23', 'cps19_lead_rating_24', 'cps19_lead_rating_25', 'cps19_lead_rating_26', 'cps19_lead_rating_27', 'cps19_lead_rating_28']
attrNamePeople=['Justin Trudeau','Andrew Scheer','Jagmeet Singh','Yves-François Blanchet','Elizabeth May','Maxime Bernier']

getMinPartyAttribute(attr,attrNamePeople,'cps19_lead_rating_min')

C:\Users\Jean\AppData\Local\Temp/ipykernel_18444/1029237248.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfPredire[newAttrName]=newAttrTab


In [32]:
#'cps19_cand_rating'
attr=['cps19_cand_rating_23', 'cps19_cand_rating_24', 'cps19_cand_rating_25', 'cps19_cand_rating_26', 'cps19_cand_rating_27', 'cps19_cand_rating_28']
attrNameParty= ['Liberal Party','Conservative Party','ndp','Bloc Québécois','Green Party',"People's Party"]

gatherPartyAttribute(attr,attrNameParty,'cps19_cand_rating_max')

C:\Users\Jean\AppData\Local\Temp/ipykernel_18444/2886697018.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfPredire[newAttrName]=newAttrTab


(17173, 2)

In [76]:
#'cps19_cand_rating_min'
attr=['cps19_cand_rating_23', 'cps19_cand_rating_24', 'cps19_cand_rating_25', 'cps19_cand_rating_26', 'cps19_cand_rating_27', 'cps19_cand_rating_28']
attrNameParty= ['Liberal Party','Conservative Party','ndp','Bloc Québécois','Green Party',"People's Party"]

getMinPartyAttribute(attr,attrNameParty,'cps19_cand_rating_min')

C:\Users\Jean\AppData\Local\Temp/ipykernel_18444/1029237248.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfPredire[newAttrName]=newAttrTab


In [33]:
def gatherPreferencePeople(attribut,attributName,newAttrName):
    newAttrTab=[]
    for index, row in dfTrain[attr].iterrows() :
        uniqueValue=True
        i=0
        while i < len(row) :
            if str(row[i]) != 'nan' :
                if uniqueValue :
                    newAttrTab.append(attributName[i])
                    uniqueValue=False
                else :
                    newAttrTab[-1]=np.nan
                    i=len(row)
                    
            i+=1
        if uniqueValue :   
            newAttrTab.append(np.nan)
    
    dfTrain[newAttrName]=newAttrTab
    
    newAttrTab=[]
    for index, row in dfPredire[attr].iterrows() :
        uniqueValue=True
        i=0
        while i < len(row) :
            if str(row[i]) != 'nan' :
                if uniqueValue :
                    newAttrTab.append(attributName[i])
                    uniqueValue=False
                else :
                    newAttrTab[-1]=np.nan
                    i=len(row)
                    
            i+=1
        if uniqueValue :   
            newAttrTab.append(np.nan)
    
    dfPredire[newAttrName]=newAttrTab
    return dfTrain[dfTrain[newAttrName]==dfTrain['vote_choice']][[newAttrName,'vote_choice']].shape

In [34]:
#'cps19_lead_strong'
attr= ['cps19_lead_strong_113','cps19_lead_strong_114','cps19_lead_strong_115','cps19_lead_strong_116','cps19_lead_strong_117','cps19_lead_strong_118','cps19_lead_strong_119','cps19_lead_strong_120']
attrNamePeople=['Justin Trudeau','Andrew Scheer','Jagmeet Singh','Yves-François Blanchet','Elizabeth May','Maxime Bernier','None of these',"Don't know/ Prefer not to answer"]
attrNameParty= ['Liberal Party','Conservative Party','ndp','Bloc Québécois','Green Party',"People's Party","Another",np.nan]

gatherPreferencePeople(attr,attrNameParty,'cps19_lead_strong_max')

C:\Users\Jean\AppData\Local\Temp/ipykernel_18444/1562130080.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfPredire[newAttrName]=newAttrTab


(8349, 2)

In [35]:
#'cps19_lead_trust'
attr= ['cps19_lead_trust_113','cps19_lead_trust_114','cps19_lead_trust_115','cps19_lead_trust_116','cps19_lead_trust_117','cps19_lead_trust_118','cps19_lead_trust_119','cps19_lead_trust_120']
attrNamePeople=['Justin Trudeau','Andrew Scheer','Jagmeet Singh','Yves-François Blanchet','Elizabeth May','Maxime Bernier','None of these',"Don't know/ Prefer not to answer"]
attrNameParty= ['Liberal Party','Conservative Party','ndp','Bloc Québécois','Green Party',"People's Party","Another",np.nan]

gatherPreferencePeople(attr,attrNameParty,'cps19_lead_trust_max')

C:\Users\Jean\AppData\Local\Temp/ipykernel_18444/1562130080.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfPredire[newAttrName]=newAttrTab


(6966, 2)

In [36]:
#'cps19_lead_cares'
attr= ['cps19_lead_cares_113','cps19_lead_cares_114','cps19_lead_cares_115','cps19_lead_cares_116','cps19_lead_cares_117','cps19_lead_cares_118','cps19_lead_cares_119','cps19_lead_cares_120']
attrNamePeople=['Justin Trudeau','Andrew Scheer','Jagmeet Singh','Yves-François Blanchet','Elizabeth May','Maxime Bernier','None of these',"Don't know/ Prefer not to answer"]
attrNameParty= ['Liberal Party','Conservative Party','ndp','Bloc Québécois','Green Party',"People's Party","Another",np.nan]

gatherPreferencePeople(attr,attrNameParty,'cps19_lead_cares_max')

C:\Users\Jean\AppData\Local\Temp/ipykernel_18444/1562130080.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfPredire[newAttrName]=newAttrTab


(7619, 2)

## Normalisation

In [37]:
# Normalisation sur [0,1] sur des valeurs sans aucun ordre, la valeur np.nan est toujours présente et égale à 0
def normaliseNaif(dfInit,dfNorm,attribut,log):
    valeurs={ np.nan : 0 }
    for elmt in dfInit[attribut]:
        if str(elmt) not in valeurs :
            valeurs[str(elmt)]=0
    nValeur=len(valeurs)-1
    i = 0
    for val in valeurs.keys():
        valeurs[val]=i/nValeur
        i+=1
    
    norm_val=[]
    for elmt in dfInit[attribut]:
        norm_val.append(valeurs[str(elmt)])
        
    char=attribut+"_norm"
    dfNorm[char]=norm_val
    
    if log : 
        print(valeurs)
        
    return valeurs

## ENTRAINEMENT

Données d'entrainement :

In [98]:
attr_selectionne=['cps19_imp_iss_party','cps19_imp_loc_iss_p','cps19_province','cps19_education','pes19_rural_urban','cps19_demsat','cps19_gender','cps19_citizenship']

attr_selectionne=['cps19_party_rating_min','cps19_lead_trust_max','cps19_issue_handle_1','cps19_issue_handle_2','cps19_issue_handle_3','cps19_issue_handle_4','cps19_issue_handle_5','cps19_issue_handle_6','cps19_issue_handle_7','cps19_issue_handle_8','cps19_lead_cares_max','cps19_lead_strong_max','cps19_party_rating_max','cps19_lead_rating_max','cps19_cand_rating_max','cps19_imp_iss_party','cps19_fed_gov_sat','cps19_imp_loc_iss_p']#75.65270188221008 %


dfX=dfTrain[attr_selectionne]
dfY=dfTrain['vote_choice']
dfX

,cps19_party_rating_min,cps19_lead_trust_max,cps19_issue_handle_1,cps19_issue_handle_2,cps19_issue_handle_3,cps19_issue_handle_4,cps19_issue_handle_5,cps19_issue_handle_6,cps19_issue_handle_7,cps19_issue_handle_8,cps19_lead_cares_max,cps19_lead_strong_max,cps19_party_rating_max,cps19_lead_rating_max,cps19_cand_rating_max,cps19_imp_iss_party,cps19_fed_gov_sat,cps19_imp_loc_iss_p
0,Conservative Party,Green Party,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Green Party,Elizabeth May,Green Party,Green Party,Not very satisfied,ndp
1,Conservative Party,Another,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Green Party,NaN,Green Party,Elizabeth May,Green Party,Green Party,Not very satisfied,ndp
2,Conservative Party,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ndp,NaN,Liberal Party,Justin Trudeau,People's Party,NaN,Fairly satisfied,NaN
3,ndp,Conservative Party,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Conservative Party,Conservative Party,Andrew Scheer,Conservative Party,Conservative Party,Not very satisfied,NaN
4,Liberal Party,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ndp,NaN,Conservative Party,Andrew Scheer,People's Party,Don't know/ Prefer not to answer,Not very satisfied,Conservative Party
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37816,People's Party,ndp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ndp,Jagmeet Singh,People's Party,Green Party,Not very satisfied,Don't know/ Prefer not to answer
37817,Conservative Party,NaN,ndp,Liberal Party,Green Party,ndp,Liberal Party,Liberal Party,Liberal Party,Liberal Party,NaN,NaN,Liberal Party,Justin Trudeau,Liberal Party,Liberal Party,Fairly satisfied,NaN
37818,ndp,Green Party,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Green Party,Green Party,Green Party,Elizabeth May,Green Party,Green Party,Not very satisfied,NaN
37820,Conservative Party,NaN,ndp,ndp,Green Party,ndp,Liberal Party,Liberal Party,Liberal Party,Conservative Party,ndp,ndp,ndp,Jagmeet Singh,Liberal Party,ndp,Fairly satisfied,NaN


Randomn Forest
(https://www.stat4decision.com/fr/foret-aleatoire-avec-python/)

In [99]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


Normalisation pour random Forest

In [100]:
dfNorm= pd.DataFrame()
for attr in attr_selectionne :
    try :
        normaliseNaif(dfX,dfNorm,attr,False)
    except:
        print(attr)

    
#dfNorm['cps19_yob_norm']=norm_val_yob

In [101]:
dfNorm

,cps19_party_rating_min_norm,cps19_lead_trust_max_norm,cps19_issue_handle_1_norm,cps19_issue_handle_2_norm,cps19_issue_handle_3_norm,cps19_issue_handle_4_norm,cps19_issue_handle_5_norm,cps19_issue_handle_6_norm,cps19_issue_handle_7_norm,cps19_issue_handle_8_norm,cps19_lead_cares_max_norm,cps19_lead_strong_max_norm,cps19_party_rating_max_norm,cps19_lead_rating_max_norm,cps19_cand_rating_max_norm,cps19_imp_iss_party_norm,cps19_fed_gov_sat_norm,cps19_imp_loc_iss_p_norm
0,0.166667,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.166667,0.166667,0.166667,0.111111,0.2,0.111111
1,0.166667,0.250,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.250,0.125,0.166667,0.166667,0.166667,0.111111,0.2,0.111111
2,0.166667,0.375,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.375,0.125,0.333333,0.333333,0.333333,0.222222,0.4,0.222222
3,0.333333,0.500,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.250,0.500000,0.500000,0.500000,0.333333,0.2,0.222222
4,0.500000,0.375,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.375,0.125,0.500000,0.500000,0.333333,0.444444,0.2,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32932,0.666667,0.625,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.666667,0.666667,0.333333,0.111111,0.2,0.555556
32933,0.166667,0.375,0.375,0.250,0.250,0.250,0.500,0.500,0.375,0.250,0.125,0.125,0.333333,0.333333,0.666667,0.555556,0.4,0.222222
32934,0.333333,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.250,0.750,0.166667,0.166667,0.166667,0.111111,0.2,0.222222
32935,0.166667,0.375,0.375,0.375,0.250,0.250,0.500,0.500,0.375,0.375,0.375,0.375,0.666667,0.666667,0.666667,0.666667,0.4,0.222222


In [102]:
x_train, x_test, y_train, y_test  = train_test_split(dfNorm, 
                                                     dfY,
                                                     test_size=0.10, 
                                                     random_state=42)

In [103]:
modele_rf = RandomForestClassifier(
     n_estimators=1000,#100 #1000
     criterion='gini',#criterion='entropy'
     max_depth=None,
     min_samples_split=50,
     min_samples_leaf=25,
     min_weight_fraction_leaf=0.0,
     max_features='auto',
     max_leaf_nodes=None,
     min_impurity_decrease=0.0,
     bootstrap=False,
     oob_score=False,
     n_jobs=None,
     random_state=None,
     verbose=0,
     warm_start=False,
     class_weight=None,
     ccp_alpha=0.0,
     max_samples=None,)

In [ ]:
modele_rf.fit(x_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
print(f"Le pourcentage de bien classés est de : {accuracy_score(y_test, modele_rf.predict(x_test))*100} %")

Max : attr_selectionne=attr_selectionne=attr_selectionne=['cps19_lead_cares_max','cps19_lead_strong_max','cps19_party_rating_max','cps19_lead_rating_max','cps19_cand_rating_max','cps19_imp_iss_party','cps19_fed_gov_sat','cps19_imp_loc_iss_p']#75.65270188221008 %

Analyse random Forest

In [46]:
x_test_np=x_test.to_numpy()
x_test_np[8]

array([0.75      , 0.25      , 0.25      , 0.75      , 0.375     ,
       0.5       , 0.5       , 0.375     , 0.25      , 0.75      ,
       0.5       , 0.33333333, 0.33333333, 0.66666667, 0.55555556,
       1.        , 0.22222222])

In [47]:
modele_rf.decision_path(x_test)

(<3294x1203166 sparse matrix of type '<class 'numpy.int64'>'
 	with 42430469 stored elements in Compressed Sparse Row format>,
 array([      0,    1171,    2404, ..., 1200746, 1201953, 1203166],
       dtype=int32))

In [96]:
pd.DataFrame(modele_rf.feature_importances_,
              index = x_train.columns, 
              columns = ["importance"]).sort_values(
     "importance", 
     ascending = False)

,importance
cps19_party_rating_max_norm,0.323932
cps19_imp_iss_party_norm,0.162373
cps19_lead_rating_max_norm,0.153113
cps19_cand_rating_max_norm,0.132970
cps19_lead_rating_min_norm,0.036681
cps19_fed_gov_sat_norm,0.033311
cps19_issue_handle_8_norm,0.022730
cps19_party_rating_min_norm,0.020951
cps19_lead_cares_max_norm,0.017855
cps19_issue_handle_1_norm,0.013185


In [49]:
pred=modele_rf.predict(x_test)

In [50]:
y_test_np=y_test.to_numpy()
for i in range(len(y_test_np)):
    if(y_test_np[])
    print(y_test_np[i])

SyntaxError: invalid syntax (Temp/ipykernel_18444/3752545181.py, line 3)

Naif Bayes

In [51]:
from sklearn.naive_bayes import BernoulliNB
# initializaing the NB
classifer = BernoulliNB()
from sklearn.naive_bayes import GaussianNB

classifer1=GaussianNB()

# training the model
classifer.fit(x_train, y_train)
classifer1.fit(x_train, y_train)

# testing the model
y_pred = classifer.predict(x_test)
y_pred1 = classifer1.predict(x_test)

In [52]:
print("Bernoulli : ",accuracy_score(y_pred, y_test))
print("Gaussian : ",accuracy_score(y_pred1, y_test))

Bernoulli :  0.2799028536733455
Gaussian :  0.6378263509411051


## Application

In [53]:
dfPredireNorm= pd.DataFrame()
for attr in attr_selectionne :
    try :
        normaliseNaif(dfPredire,dfPredireNorm,attr,False)
    except:
        print(attr)

In [54]:
dfPredire['vote_choice']=modele_rf.predict(dfPredireNorm)
dfPredire[['Unnamed: 0','vote_choice']]


C:\Users\Jean\AppData\Local\Temp/ipykernel_18444/4189917358.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfPredire['vote_choice']=modele_rf.predict(dfPredireNorm)


,Unnamed: 0,vote_choice
7,8,Green Party
22,23,Liberal Party
45,46,Conservative Party
59,60,Liberal Party
63,64,Conservative Party
...,...,...
37791,37792,Don't know/ Prefer not to answer
37792,37793,Bloc Qu<e9>b<e9>cois
37793,37794,Liberal Party
37797,37798,Conservative Party


In [55]:
f=open('prediction.txt','w')
for pred in dfPredire[['Unnamed: 0','vote_choice']].iterrows():
    string=str(pred[1][0])+" "+pred[1][1]+"\n"
    #print(string)
    f.write(string)
f.close()

In [56]:
tab=[]
ftest=open('exemple.txt','r')
for line in ftest.readlines():
    #print(line.split())
    tab.append(int(line.split()[0]))
ftest.close()

In [57]:
tab=np.sort(tab)
f=open('prediction_restricted.txt','w')
compteur=0
for pred in dfPredire[['Unnamed: 0','vote_choice']].iterrows():
    print(pred[1][0]," : ",tab[compteur])
    if(pred[1][0]-1 in tab):
        string=str(pred[1][0])+" "+pred[1][1]+"\n"
        #print(string)
        f.write(string)
        compteur+=1
f.close()
compteur

8  :  7
23  :  22
46  :  45
60  :  59
64  :  63
78  :  77
92  :  91
94  :  93
133  :  132
140  :  139
146  :  145
165  :  164
169  :  168
171  :  170
176  :  175
182  :  181
183  :  182
184  :  183
191  :  214
194  :  214
209  :  214
215  :  214
220  :  219
223  :  222
231  :  230
234  :  233
236  :  235
247  :  246
261  :  260
264  :  266
267  :  266
271  :  284
285  :  284
291  :  290
296  :  298
299  :  298
302  :  301
309  :  308
329  :  328
330  :  329
339  :  338
347  :  346
355  :  354
357  :  356
362  :  361
366  :  365
368  :  369
370  :  369
395  :  394
396  :  395
403  :  412
405  :  412
413  :  412
431  :  430
447  :  446
467  :  466
469  :  484
475  :  484
480  :  484
485  :  484
494  :  498
499  :  498
502  :  509
510  :  509
520  :  525
526  :  525
527  :  526
534  :  533
547  :  546
556  :  555
557  :  556
569  :  568
579  :  578
586  :  585
602  :  601
619  :  618
631  :  630
635  :  634
636  :  635
641  :  642
643  :  642
644  :  643
655  :  667
668  :  667
676  :  67

5262  :  5261
5280  :  5279
5288  :  5287
5309  :  5308
5316  :  5315
5330  :  5329
5334  :  5336
5337  :  5336
5346  :  5345
5348  :  5347
5352  :  5351
5361  :  5360
5373  :  5372
5376  :  5375
5383  :  5389
5389  :  5389
5390  :  5389
5392  :  5398
5399  :  5398
5404  :  5403
5406  :  5405
5409  :  5408
5422  :  5421
5424  :  5423
5436  :  5435
5454  :  5453
5458  :  5457
5467  :  5472
5473  :  5472
5476  :  5475
5482  :  5481
5487  :  5486
5490  :  5489
5495  :  5494
5510  :  5509
5517  :  5518
5519  :  5518
5522  :  5521
5530  :  5535
5536  :  5535
5540  :  5539
5553  :  5552
5566  :  5565
5598  :  5597
5611  :  5610
5623  :  5624
5625  :  5624
5630  :  5629
5637  :  5636
5640  :  5639
5643  :  5642
5655  :  5654
5669  :  5674
5675  :  5674
5686  :  5685
5693  :  5692
5699  :  5698
5701  :  5705
5706  :  5705
5710  :  5709
5712  :  5711
5749  :  5748
5752  :  5766
5767  :  5766
5773  :  5772
5780  :  5779
5793  :  5792
5801  :  5800
5811  :  5810
5812  :  5812
5813  :  5812
5814  

13630  :  13636
13637  :  13636
13642  :  13648
13649  :  13648
13654  :  13653
13673  :  13678
13679  :  13678
13680  :  13679
13681  :  13680
13684  :  13683
13685  :  13684
13687  :  13686
13714  :  13729
13730  :  13729
13731  :  13730
13735  :  13734
13736  :  13735
13738  :  13737
13741  :  13745
13746  :  13745
13759  :  13759
13760  :  13759
13763  :  13762
13768  :  13770
13771  :  13770
13772  :  13783
13783  :  13783
13784  :  13783
13797  :  13796
13803  :  13804
13805  :  13804
13806  :  13826
13827  :  13826
13836  :  13852
13853  :  13852
13861  :  13860
13862  :  13861
13865  :  13864
13873  :  13872
13883  :  13882
13889  :  13888
13890  :  13903
13904  :  13903
13914  :  13913
13923  :  13922
13924  :  13923
13928  :  13932
13933  :  13932
13937  :  13936
13938  :  13938
13939  :  13938
13944  :  13943
13948  :  13947
13956  :  13956
13957  :  13956
13960  :  13959
13963  :  13962
13968  :  13967
13972  :  13999
13994  :  13999
14000  :  13999
14011  :  14010
14016  :

17669  :  17668
17676  :  17675
17680  :  17699
17696  :  17699
17700  :  17699
17708  :  17707
17718  :  17717
17721  :  17720
17724  :  17723
17753  :  17752
17773  :  17772
17793  :  17792
17813  :  17813
17814  :  17813
17820  :  17819
17831  :  17840
17841  :  17840
17853  :  17852
17855  :  17870
17871  :  17870
17876  :  17889
17879  :  17889
17890  :  17889
17906  :  17905
17920  :  17919
17933  :  17943
17944  :  17943
17947  :  17946
17959  :  17958
17967  :  17966
17973  :  17972
17980  :  17979
17990  :  17989
17995  :  18005
17999  :  18005
18006  :  18005
18011  :  18031
18032  :  18031
18033  :  18033
18034  :  18033
18035  :  18034
18036  :  18035
18042  :  18045
18046  :  18045
18057  :  18056
18062  :  18062
18063  :  18062
18064  :  18063
18071  :  18070
18083  :  18082
18101  :  18111
18112  :  18111
18113  :  18112
18116  :  18115
18126  :  18125
18135  :  18134
18143  :  18142
18146  :  18145
18156  :  18155
18163  :  18176
18177  :  18176
18179  :  18178
18194  :

21937  :  21969
21970  :  21969
21975  :  21974
21998  :  21997
22001  :  22000
22010  :  22009
22019  :  22018
22030  :  22029
22040  :  22039
22044  :  22043
22048  :  22047
22050  :  22049
22052  :  22051
22063  :  22072
22073  :  22072
22084  :  22083
22099  :  22098
22108  :  22107
22111  :  22110
22123  :  22122
22125  :  22124
22140  :  22142
22143  :  22142
22144  :  22143
22148  :  22147
22151  :  22153
22154  :  22153
22159  :  22158
22166  :  22165
22173  :  22172
22175  :  22199
22179  :  22199
22192  :  22199
22200  :  22199
22206  :  22205
22223  :  22222
22229  :  22228
22232  :  22231
22248  :  22248
22249  :  22248
22261  :  22260
22285  :  22284
22290  :  22289
22299  :  22298
22302  :  22301
22310  :  22310
22311  :  22310
22315  :  22314
22318  :  22317
22328  :  22327
22330  :  22329
22340  :  22345
22346  :  22345
22377  :  22384
22385  :  22384
22387  :  22401
22402  :  22401
22404  :  22408
22409  :  22408
22411  :  22410
22424  :  22423
22436  :  22435
22440  :

30247  :  30246
30254  :  30253
30256  :  30256
30257  :  30256
30262  :  30261
30267  :  30266
30275  :  30274
30276  :  30283
30283  :  30283
30284  :  30283
30286  :  30301
30302  :  30301
30309  :  30308
30314  :  30313
30317  :  30316
30318  :  30317
30324  :  30323
30329  :  30328
30335  :  30334
30345  :  30378
30371  :  30378
30379  :  30378
30380  :  30379
30384  :  30383
30386  :  30385
30391  :  30390
30393  :  30392
30398  :  30404
30405  :  30404
30407  :  30407
30408  :  30407
30412  :  30432
30422  :  30432
30423  :  30432
30427  :  30432
30433  :  30432
30434  :  30433
30436  :  30435
30445  :  30444
30469  :  30468
30497  :  30496
30498  :  30521
30522  :  30521
30524  :  30548
30549  :  30548
30557  :  30556
30564  :  30563
30567  :  30566
30607  :  30606
30609  :  30612
30613  :  30612
30615  :  30614
30618  :  30617
30620  :  30619
30625  :  30624
30637  :  30636
30644  :  30643
30671  :  30670
30676  :  30675
30679  :  30678
30690  :  30689
30695  :  30694
30704  :

34299  :  34298
34302  :  34305
34306  :  34305
34315  :  34316
34317  :  34316
34327  :  34331
34328  :  34331
34332  :  34331
34336  :  34335
34341  :  34340
34343  :  34342
34348  :  34347
34365  :  34375
34366  :  34375
34376  :  34375
34385  :  34388
34389  :  34388
34399  :  34398
34401  :  34400
34404  :  34403
34409  :  34408
34437  :  34444
34445  :  34444
34451  :  34472
34473  :  34472
34475  :  34474
34476  :  34475
34479  :  34478
34483  :  34482
34521  :  34520
34522  :  34534
34535  :  34534
34547  :  34546
34558  :  34557
34594  :  34616
34617  :  34616
34626  :  34625
34630  :  34635
34636  :  34635
34639  :  34638
34647  :  34646
34652  :  34651
34664  :  34663
34681  :  34680
34696  :  34695
34698  :  34697
34719  :  34718
34740  :  34739
34744  :  34743
34749  :  34758
34750  :  34758
34757  :  34758
34759  :  34758
34767  :  34766
34777  :  34776
34786  :  34785
34788  :  34799
34800  :  34799
34809  :  34808
34812  :  34811
34813  :  34812
34820  :  34819
34843  :

3659